In [10]:
import json
import os

def convert_txt_to_json(txt_file, output_dir):
    # 打开并读取 txt 文件
    with open(txt_file, 'r') as file:
        lines = file.readlines()
    
    # 第一行为关键点名称及顺序
    headers = lines[0].strip().split(';')  # 去掉换行符并按 ";" 分割
    keypoints = [header.split('.')[0] for header in headers[::3]]  # 提取关键点名称

    # 逐行处理关键点坐标
    for i, line in enumerate(lines[1:], start=1):
        data = line.strip().split(';')  # 每行按 ";" 分割
        data = data[:-1]
        coordinates = [list(map(lambda x: float(x.replace(',', '.')), data[j:j+3])) 
                       for j in range(0, len(data), 3)]  # 每三个为一组，处理为浮点数
    

        # 将关键点信息打包为 COCO 格式
        keypoints_flat = [item for sublist in coordinates for item in sublist]  # 展平为1D列表

                # 0: "nose",
                # 1: "left_eye",
                # 2: "right_eye",
                # 3: "left_ear",
                # 4: "right_ear",
                # 5: "left_shoulder",
                # 6: "right_shoulder",
                # 7: "left_elbow",
                # 8: "right_elbow",
                # 9: "left_wrist",
                # 10: "right_wrist",
                # 11: "left_hip",
                # 12: "right_hip",
                # 13: "left_knee",
                # 14: "right_knee",
                # 15: "left_ankle",
                # 16: "right_ankle"

                # //     {0,  "Nose"},
                # //     {1,  "Neck"},
                # //     {2,  "RShoulder"},
                # //     {3,  "RElbow"},
                # //     {4,  "RWrist"},
                # //     {5,  "LShoulder"},
                # //     {6,  "LElbow"},
                # //     {7,  "LWrist"},
                # //     {8,  "RHip"},
                # //     {9,  "RKnee"},
                # //     {10, "RAnkle"},
                # //     {11, "LHip"},
                # //     {12, "LKnee"},
                # //     {13, "LAnkle"},
                # //     {14, "REye"},
                # //     {15, "LEye"},
                # //     {16, "REar"},
                # //     {17, "LEar"}

        related_sol = [0,15,14,17,16,5,2,6,3,7,4,11,8,12,9,13,10]
        keypoints_coco_17 = list()

        for ind in related_sol: 
            keypoints_coco_17.append(keypoints_flat[ind*3 + 0])
            keypoints_coco_17.append(keypoints_flat[ind*3 + 1])
            keypoints_coco_17.append(keypoints_flat[ind*3 + 2])

        # 构建 COCO 格式数据
        coco_format = {
            "images": [
                {
                    "id": 1,
                    "file_name": 'image',
                    "width": 640,
                    "height": 480
                }
            ],
            "annotations": [
                {
                    "id": 1,
                    "image_id": 1,
                    "category_id": 1,  # 假设只有 "person" 类别
                    "keypoints": keypoints_coco_17,
                    "num_keypoints": 17,  # 可见关键点数量
                    "bbox": [50, 200, 150, 350],  # 示例边界框 [x_min, y_min, width, height]
                    "area": 150 * 350,  # bbox面积 (width * height)
                    "iscrowd": 0
                }
            ],
            "categories": [
                {
                    "id": 1,
                    "name": "person",
                    "supercategory": "person",
                    "keypoints": [
                        "nose", "left_eye", "right_eye", "left_ear", "right_ear",
                        "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
                        "left_wrist", "right_wrist", "left_hip", "right_hip",
                        "left_knee", "right_knee", "left_ankle", "right_ankle"
                    ],
                    "skeleton": [
                        [1, 2], [1, 3], [2, 4], [3, 5], [6, 7],
                        [6, 8], [7, 9], [8, 10], [9, 11], [12, 13],
                        [12, 14], [13, 15], [14, 16], [15, 17]
                    ]
                }
            ]
        }
        
        # 保存为 JSON 文件
        output_path = os.path.join(output_dir,(str(i).zfill(6) + '.json'))
        with open(output_path, 'w') as json_file:
            json.dump(coco_format, json_file, indent=4)
        # print(f"Frame {i} saved to {output_path}")




In [11]:
# 使用示例
txt_file_path = "/home/sparkle79/pypro/PoVER/swimXYZ/2D/free_wl_p3/gt/2D_cam.txt"  # 替换为你的 txt 文件路径
output_directory = "/home/sparkle79/pypro/PoVER/swimXYZ/2D/free_wl_p3/gt/jsons"  # 替换为你的输出文件夹路径

import os
os.makedirs(output_directory, exist_ok=True)  # 如果输出文件夹不存在则创建

convert_txt_to_json(txt_file_path, output_directory)